In [247]:
import string
import numpy as np
import pandas as pd
from nltk.tokenize import sent_tokenize, word_tokenize

# Esercizio 2.1 - Text summarization estrattivo

Per questo esercizio è stato scelto come corpus la pagina di Wikipedia sul _Natural Language Processing_ (<https://en.wikipedia.org/wiki/Natural_language_processing>).
Il testo è stato recuperato utilizzando _SketchEngine_.

L'obiettivo è quello di implementare un sistema di __text summarization estrattivo__ che riduca il numero di frasi del documento senza tralasciare informazioni importanti.

Carichiamo il corpus e creiamo un array con le singole frasi.

In [248]:
all_sentences = np.array([])
for line in open('utils/wikipedia_nlp_page.txt', 'r').readlines():
    all_sentences = np.append(all_sentences, sent_tokenize(line))
all_sentences[:5]

array(['Natural language processing',
       'This article is about natural language processing done by computers.',
       'For the natural language processing done by the human brain, see Language processing in the brain.',
       'Natural language processing (NLP) is a subfield of linguistics, computer science, and artificial intelligence concerned with the interactions between computers and human language, in particular how to program computers to process and analyze large amounts of natural language data.',
       'The goal is a computer capable of "understanding" the contents of documents, including the contextual nuances of the language within them.'],
      dtype='<U423')

Creiamo una copia dell'array per non modificare le frasi originali in fase di preprocessing (stopwards e punct removal ...) e poterle così recuperare intatte in fase di costruzione del riassunto.

In [249]:
summary = np.copy(all_sentences)

Effettuiamo ora preprocessing sulle frasi dell'array copia.

In [250]:
stopwords = []
for line in open("utils/stop_words_FULL.txt", 'r').readlines():
    stopwords.append(line.rstrip('\n'))
stopwords = np.array(stopwords)


def preprocessing(s):
    """
    Do some preprocessing operations on the string.

    :param s: the string

    :return: the preprocessed string
    """
    # Lowercasing
    s = s.lower()
    # Punct removal
    s = s.translate(str.maketrans('', '', string.punctuation))
    # Stopword removal
    s = ' '.join([word for word in s.split() if word not in stopwords])
    return s

In [251]:
vect_preprocessing = np.vectorize(preprocessing) # vettorizziamo la funzione in modo da applicarla a tutto l'array di frasi in maniera efficiente
summary = pd.DataFrame(vect_preprocessing(summary), columns=['sentences'])
summary.head()

,sentences
0,natural language processing
1,article natural language processing computers
2,natural language processing human brain langua...
3,natural language processing nlp subfield lingu...
4,goal computer capable understanding contents d...


Per valutare le frasi e selezionarne le più importanti occorre avere un punteggio per ognuna di esse. Utilizziamo l'informazione statistica derivante dal testo, calcolando per ogni frase uno __score__ basato sulla frequenza delle singole parole nell'intero testo: ad ogni frase verrà quindi assegnato un punteggio pari alla somma degli score di tutte le parole che la compongono.

Per prima cosa contiamo le __frequenze__ di tutte le parole nel testo, al netto delle _stopwords_ che vengono eliminate in fase di preprocessing, rendendo la computazione più snella.

In [252]:
def get_frequencies(sents):
    """
    Function to get the frequency of the words in the given sentences.

    :param sents: a list of sentences

    :return: a dictionary of words with their frequency
    """
    words = []
    # prendiamo le singole parole
    for s in sents:
        words += word_tokenize(s)
    # contiamo le parole in un dizionario
    counts = dict.fromkeys(words, 0) # inizializziamo i conteggi di tutte le parole a 0
    for w in words:
        counts[w] += 1
    for k in counts.keys():
        counts[k] = counts[k] / max(counts.values()) # scaliamo i valori rendendoli tutti <=1
    return counts

In [253]:
frequencies = get_frequencies(summary['sentences'])
list(frequencies.items())[:5]

[('natural', 0.6212121212121212),
 ('language', 1.0),
 ('processing', 1.0),
 ('article', 0.07142857142857142),
 ('computers', 0.10714285714285714)]

Sommiamo adesso gli score delle singole parole di ogni frase, in modo da avere un punteggio totale per ognuna di esse.

In [254]:
def compute_score(s, freq):
    """
    Calculate the score of a sentence based on the frequency of its words.

    :param s: the sentence
    :param freq: a dictionary with the words' frequencies score in the values

    :return: the score of the sentence
    """
    score = 0
    for w in s.split():
        score += freq[w]
    return score


scores = []
for s in summary['sentences']:
    scores.append(compute_score(s, frequencies))
summary['score'] = scores
summary = summary.sort_values(by='score', ascending=False)
summary.head()

,sentences,score
3,natural language processing nlp subfield lingu...,9.857809
192,interest increasingly abstract cognitive aspec...,9.419519
233,chomskyan linguistics encourages investigation...,9.185223
67,partofspeech tagging introduced hidden markov ...,8.351981
9,premise symbolic nlp wellsummarized john searl...,8.313520


Ora che abbiamo tutti gli score delle frasi possiamo procedere alla composizione del riassunto.
Per cominciare stabiliamo di quanto ridurre il documento.

In [255]:
compression_rate = 0.3
document_len = len(all_sentences) # 314
summary_len = int(document_len * compression_rate)
print(f'Il documento verrà riassunto in {summary_len}/{document_len} frasi')

Il documento verrà riassunto in 94/314 frasi


Prendiamo quindi le prime $n$ frasi con lo score maggiore, le riordiniamo in base al loro ordine di uscita nel documento originale ed usiamo i loro indici per ricondurci alle frasi originali.

In [256]:
# prendiamo le frasi rilevanti e le ordiniamo
indexes = summary.index[:summary_len].sort_values()
# passiamo alle frasi originali
summary = all_sentences[indexes]
# stampiamo un'anteprima del riassunto
summary[:5]

array(['For the natural language processing done by the human brain, see Language processing in the brain.',
       'Natural language processing (NLP) is a subfield of linguistics, computer science, and artificial intelligence concerned with the interactions between computers and human language, in particular how to program computers to process and analyze large amounts of natural language data.',
       "The premise of symbolic NLP is well-summarized by John Searle's Chinese room experiment: Given a collection of rules (e.g., a Chinese phrasebook, with questions and matching answers), the computer emulates natural language understanding (or other NLP tasks) by applying those rules to the data it confronts.",
       'Little further research in machine translation was conducted until the late 1980s when the first statistical machine translation systems were developed.',
       '1960s: Some notably successful natural language processing systems developed in the 1960s were SHRDLU, a natur

Salviamo il riassunto completo in un file di testo per avere una visualizzazione migliore.

In [257]:
np.savetxt(r'output/summary.txt', summary, fmt='%s')